In [4]:
import cv2
import numpy as np
import pandas as pd
import scipy
from scipy.misc import imread
import cPickle as pickle
import random
import os
import matplotlib.pyplot as plt
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip
import shutil
from PIL import Image
import av
from tqdm import tqdm
%matplotlib inline

In [9]:
def rgb_to_bgr(image):
    return image[:, :, ::-1].copy() 

def preprocess_image(image, resize):
    image = image.resize(resize)
    return image

def histogram(image, mask, bins):
    # extract a 3D color histogram from the masked region of the
    # image, using the supplied number of bins per channel
    
    hist = cv2.calcHist([image], [0, 1, 2], mask, bins,
        [0, 256, 0, 256, 0, 256])

    hist = cv2.normalize(hist, hist).flatten()

    return hist
    
def local_histogram_features(image, bins=[5, 6, 6]):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    features = []

    # grab the dimensions and compute the center of the image
    (h, w) = image.shape[:2]
    (cX, cY) = (int(w * 0.5), int(h * 0.5))

    # divide the image into four rectangles/segments (top-left,
    # top-right, bottom-right, bottom-left)
    segments = [(0, cX, 0, cY), (cX, w, 0, cY), (cX, w, cY, h),
        (0, cX, cY, h)]

    # construct an elliptical mask representing the center of the
    # image
    (axesX, axesY) = (int(w * 0.75) // 2, int(h * 0.75) // 2)
    ellipMask = np.zeros(image.shape[:2], dtype = "uint8")
    cv2.ellipse(ellipMask, (cX, cY), (axesX, axesY), 0, 0, 360, 255, -1)

    # loop over the segments
    for (startX, endX, startY, endY) in segments:
        # construct a mask for each corner of the image, subtracting
        # the elliptical center from it
        cornerMask = np.zeros(image.shape[:2], dtype = "uint8")
        cv2.rectangle(cornerMask, (startX, startY), (endX, endY), 255, -1)
        cornerMask = cv2.subtract(cornerMask, ellipMask)

        # extract a color histogram from the image, then update the
        # feature vector
        hist = histogram(image, cornerMask, bins=bins)
        features.extend(hist)

    # extract a color histogram from the elliptical region and
    # update the feature vector
    hist = histogram(image, ellipMask, bins=bins)
    features.extend(hist)

    # return the feature vector
    return np.array(features)

In [10]:
def get_video_rows(path, feature_extractor, resize=(300, 300), write_frames_dir=None):
    frames_path = None
    if write_frames_dir:
        frames_path = os.path.join(write_frames_dir, os.path.basename(path)+'_frames')
        if os.path.exists(frames_path):
            shutil.rmtree(frames_path)
        os.makedirs(frames_path)
    
    rows = []
    video_name = os.path.basename(path)

    container = av.open(path)

    stream = container.streams.video[0]
    stream.codec_context.skip_frame = 'NONKEY'
    for frame in container.decode(stream):
        img = frame.to_image()
        img = preprocess_image(img, resize=resize)
        features = feature_extractor(rgb_to_bgr(np.array(img)))
        row = [path, frame.time] #metadata, features
        for i in range(len(features)):
            row.append(features[i])
        rows.append(tuple(row))
        if frames_path:
            fpath = os.path.join(frames_path, '{}.jpg'.format(str(frame.pts)))
            img.save(fpath)
    return rows

def get_dataframe(fpaths, feature_extractor, write_frames_dir=None):
    rows = []
    i = 0
    for fpath in fpaths:
        video_rows = get_video_rows(fpath, feature_extractor=feature_extractor, write_frames_dir=write_frames_dir)
        rows += video_rows
        i+=1
        print 'Done ' + str(round(float(i)/len(fpaths), 2))
    
    cols = ['video_path', 'frame_time']
    for i in range(len(rows[0])-2):
        cols.append('x_'+str(i))
    df = pd.DataFrame(rows, columns=cols, index=range(len(rows)))    
    return df

In [11]:
vid_path = './tmp/1942 - Saludos Amigos.m4v_subclips/(70, 80).m4v'

df = get_dataframe([vid_path], local_histogram_features)

Done 1.0


In [12]:
df.head()

,video_path,frame_time,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,...,x_890,x_891,x_892,x_893,x_894,x_895,x_896,x_897,x_898,x_899
0,"./tmp/1942 - Saludos Amigos.m4v_subclips/(70, ...",1.154,0.999998,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"./tmp/1942 - Saludos Amigos.m4v_subclips/(70, ...",2.072,0.999998,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"./tmp/1942 - Saludos Amigos.m4v_subclips/(70, ...",7.119,0.001728,0.001037,0.000000,0.000000,0.0,0.000000,0.012266,0.009501,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"./tmp/1942 - Saludos Amigos.m4v_subclips/(70, ...",8.787,0.019906,0.009621,0.006635,0.002654,0.0,0.004976,0.307211,0.036494,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
